In [ ]:
import json
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 파일 경로 - 아직 팀원 데이터 취합x 
file_path = './one_all/before/tagged_one_all1_before.json'

# koelectra tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\KDP-23\anaconda3\envs\imc_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 
class KoNERDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [3]:
# ner data 불러오기
def load_ner_data(file_path):
    """JSON 파일에서 NER 데이터 로드"""
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [4]:
def prepare_koelectra_ner_data(data, tokenizer, max_length=128):
    """KoElectra용 NER 데이터 준비"""
    tokens = [d['tokens'] for d in data]
    labels = [d['ner_tags'] for d in data]

    # KoElectra 토크나이저로 인코딩
    encodings = tokenizer(
        tokens, 
        is_split_into_words=True, 
        padding=True, 
        truncation=True, 
        max_length=max_length,
        return_tensors='pt'
    )

    # NER 라벨 처리
    new_labels = []
    for i, label in enumerate(labels):
        word_ids = encodings.word_ids(batch_index=i)
        label_ids = [-100] * len(word_ids)  # 초기값 -100으로 설정
        
        # 개선된 라벨 매핑 로직
        for word_idx, tag in enumerate(label):
            # word_ids에서 해당 word_idx의 첫 번째 인덱스 찾기
            indices = [j for j, w_id in enumerate(word_ids) if w_id == word_idx]
            
            if indices:
                # 첫 번째 인덱스에 태그 할당
                label_ids[indices[0]] = tag
        
        new_labels.append(label_ids)

    return encodings, new_labels


In [5]:
def debug_tokenization(tokens, tokenizer):
    """토크나이징 과정 디버깅"""
    print("원본 토큰:", tokens)
    
    # 토큰화 결과 확인
    encoded = tokenizer(
        tokens, 
        is_split_into_words=True, 
        return_tensors='pt'
    )
    
    # 단어 ID 출력
    print("Word IDs:", encoded.word_ids(batch_index=0))
    
    # 디코딩된 토큰 확인
    decoded_tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])
    print("디코딩된 토큰:", decoded_tokens)

In [6]:
def train_koelectra_ner_model(train_dataset, num_labels=7, epochs=10):
    """KoElectra NER 모델 학습"""
    # KoElectra 모델 초기화
    model = AutoModelForTokenClassification.from_pretrained(
        "monologg/koelectra-base-v3-discriminator", 
        num_labels=num_labels)

    # 학습
    model.to(device)
    model.train()

    # optim 설정 
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    # 학습 루프
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_dataset:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids, 
                attention_mask=attention_mask, 
                labels=labels
            )
            
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {total_loss/len(train_dataset):.4f}")

    return model

In [7]:
def evaluate_koelectra_ner_model(model, val_dataset):
    """모델 평가"""
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_dataset:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            outputs = model(
                input_ids, 
                attention_mask=attention_mask
            )
            
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            # 유효한 예측과 레이블만 수집
            for pred, label in zip(predictions, labels):
                mask = label != -100
                all_preds.extend(pred[mask].cpu().numpy())
                all_labels.extend(label[mask].numpy())

    # 상세 분류 보고서 출력
    print(classification_report(all_labels, all_preds))

In [ ]:
def main():
    # 데이터 로드
    data = load_ner_data(file_path)

    # 데이터 분할 (학습:검증 = 8:2)
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    # 토큰화 결과 확인
    debug_tokenization(train_data[0]['tokens'], tokenizer)

    # 데이터 준비
    train_encodings, train_labels = prepare_koelectra_ner_data(train_data, tokenizer)
    val_encodings, val_labels = prepare_koelectra_ner_data(val_data, tokenizer)

    # 데이터셋 및 데이터로더 생성
    train_dataset = DataLoader(
        KoNERDataset(train_encodings, train_labels), 
        batch_size=8, 
        
        shuffle=True
    )
    val_dataset = DataLoader(
        KoNERDataset(val_encodings, val_labels), 
        batch_size=8, 
        shuffle=False
    )

    # 모델 학습
    model = train_koelectra_ner_model(train_dataset)

    # 모델 평가
    evaluate_koelectra_ner_model(model, val_dataset)

    # 모델 저장
    model.save_pretrained('./MODEL/koelectra_v3_ner_model2')
    tokenizer.save_pretrained('./MODEL/koelectra_v3_ner_model2')
 
if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: './tagged_one_all.json'

In [ ]:
# # 저장된 토크나이저 로드
# tokenizer = AutoTokenizer.from_pretrained('./MODEL/koelectra_v3_ner_model2')

# 입력 문장
# sentence = "중화루에서 깐풍기 소자 하나랑 나홀로 세트 b번 두개, 야끼우동 세개 주문할게요"

# # 토큰화
# encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
# input_ids = encoding['input_ids']
# attention_mask = encoding['attention_mask']

In [ ]:
# # 한글자씩 임의로 토큰화
# sentence = "중화루에서 깐풍기 소자 하나랑 나홀로 세트 b번 두개, 야끼우동 세개 주문할게요"
# def new_tokenizer(sentence):
#     text = []
#     a = list(sentence)
#     for b in a:
#         if b == ' ':
#             pass
#         else:
#             text.append(b)
#     return text

# tokenizer = new_tokenizer(sentence)
# print(tokenizer)


['중', '화', '루', '에', '서', '깐', '풍', '기', '소', '자', '하', '나', '랑', '나', '홀', '로', '세', '트', 'b', '번', '두', '개', ',', '야', '끼', '우', '동', '세', '개', '주', '문', '할', '게', '요']


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
# 기존 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('./MODEL/koelectra_v3_ner_model2')

# 입력 문장
sentence = "중화루에서 깐풍기 소자 하나랑 나홀로 세트 b번 두개, 야끼우동 세개 주문할게요"

# 기본 토큰화 및 인코딩
encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

# input_ids와 attention_mask 그대로 확인
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

print("기존 encoding:", encoding)
print("기존 input_ids:", input_ids)
print("기존 attention_mask:", attention_mask)

# 기본 토큰화를 한글자 단위로 후처리
tokens = tokenizer.tokenize(sentence)
list_text = []
text_token = []
print("tokens:",tokens)
for token in tokens:
    if token.startswith("##"):  # 특수 기호 제거
        list_text[-1] += token[2:]
    else:
        list_text.append(list(token))  

for a in list_text:
    text_token.extend(a)

# 한글자 단위 결과 출력
print("한글자 단위 토큰:", text_token)

# input_ids를 한글자 단위로 다시 매핑 (필요 시 직접 ID를 생성)
char_input_ids = [tokenizer.convert_tokens_to_ids(t) for t in text_token]
print("한글자 단위 input_ids:", char_input_ids)

c:\Users\KDP-23\anaconda3\envs\imc_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


기존 encoding: {'input_ids': tensor([[    2, 11945,  4182,  4073,  4129,  2150,  4103,  4031, 23140,  6272,
          4294,  2236,  4392,  4239,  8063,    70,  4467,  2419,  4217,    16,
          3102,  4887,  4180,  4089,  2926,  4217,  7443,  4519, 11843,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}
기존 input_ids: tensor([[    2, 11945,  4182,  4073,  4129,  2150,  4103,  4031, 23140,  6272,
          4294,  2236,  4392,  4239,  8063,    70,  4467,  2419,  4217,    16,
          3102,  4887,  4180,  4089,  2926,  4217,  7443,  4519, 11843,     3]])
기존 attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])
tokens: ['중화', '##루', '##에', '##서', '깐', '##풍', '##기', '소자', '하나', '##랑', '나', '##홀', '##로', 

In [ ]:
# input_ids를 torch 텐서로 변환
char_input_ids = torch.tensor([char_input_ids])  # 2차원 텐서로 변환
attention_mask = torch.ones_like(char_input_ids)  # 모든 토큰에 대해 마스크 생성

# 저장된 모델 로드
model = AutoModelForTokenClassification.from_pretrained('./MODEL/koelectra_v3_ner_model2')
model.eval()

# 추론
with torch.no_grad():
    outputs = model(input_ids=char_input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # 가장 높은 확률을 가진 클래스 선택

In [ ]:
print(text_token)
print(predictions[0])

['중', '화', '루', '에', '서', '깐', '풍', '기', '소', '자', '하', '나', '랑', '나', '홀', '로', '세', '트', 'b', '번', '두', '개', ',', '야', '끼', '우', '동', '세', '개', '주', '문', '할', '게', '요']
tensor([0, 1, 2, 0, 0, 3, 4, 4, 4, 4, 5, 6, 0, 3, 4, 4, 4, 4, 4, 4, 5, 6, 0, 3,
        4, 4, 4, 5, 6, 0, 0, 0, 0, 0])


In [ ]:
# 토큰과 예측된 레이블 연결
# tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
predicted_labels = predictions[0].cpu().numpy()

# 결과 출력
for token, label in zip(text_token, predicted_labels):
    print(f"Token: {token} Label: {label}")

Token: 중 Label: 0
Token: 화 Label: 1
Token: 루 Label: 2
Token: 에 Label: 0
Token: 서 Label: 0
Token: 깐 Label: 3
Token: 풍 Label: 4
Token: 기 Label: 4
Token: 소 Label: 4
Token: 자 Label: 4
Token: 하 Label: 5
Token: 나 Label: 6
Token: 랑 Label: 0
Token: 나 Label: 3
Token: 홀 Label: 4
Token: 로 Label: 4
Token: 세 Label: 4
Token: 트 Label: 4
Token: b Label: 4
Token: 번 Label: 4
Token: 두 Label: 5
Token: 개 Label: 6
Token: , Label: 0
Token: 야 Label: 3
Token: 끼 Label: 4
Token: 우 Label: 4
Token: 동 Label: 4
Token: 세 Label: 5
Token: 개 Label: 6
Token: 주 Label: 0
Token: 문 Label: 0
Token: 할 Label: 0
Token: 게 Label: 0
Token: 요 Label: 0


- - -

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
# 기존 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('./MODEL/koelectra_v3_ner_model2')

# 입력 문장
sentence = "치밥의달인에서 불향바베큐 치밥도시락 3개, 담백한 숯불간장 치밥 2개 주문할게"

# 기본 토큰화 및 인코딩
encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

# input_ids와 attention_mask 그대로 확인
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

print("기존 encoding:", encoding)
print("기존 input_ids:", input_ids)
print("기존 attention_mask:", attention_mask)

# 기본 토큰화를 한글자 단위로 후처리
tokens = tokenizer.tokenize(sentence)
list_text = []
text_token = []
print("tokens:",tokens)
for token in tokens:
    if token.startswith("##"):  # 특수 기호 제거
        list_text[-1] += token[2:]
    else:
        list_text.append(list(token))  

for a in list_text:
    text_token.extend(a)

# 한글자 단위 결과 출력
print("한글자 단위 토큰:", text_token)

# input_ids를 한글자 단위로 다시 매핑 (필요 시 직접 ID를 생성)
char_input_ids = [tokenizer.convert_tokens_to_ids(t) for t in text_token]
print("한글자 단위 input_ids:", char_input_ids)

기존 encoding: {'input_ids': tensor([[    2,  3496,  4717,  4234,  4293,  4139,  4073,  4129,  2803,  4160,
          4178,  4295,  4724,  3496,  4717,  4086,  4114,  4306,    23,  4217,
            16, 18417,  4283, 27487,  4629,  4048,  3496,  4717,    22,  4217,
          7443,  4519,  4325,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
기존 input_ids: tensor([[    2,  3496,  4717,  4234,  4293,  4139,  4073,  4129,  2803,  4160,
          4178,  4295,  4724,  3496,  4717,  4086,  4114,  4306,    23,  4217,
            16, 18417,  4283, 27487,  4629,  4048,  3496,  4717,    22,  4217,
          7443,  4519,  4325,     3]])
기존 attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 

In [ ]:
# input_ids를 torch 텐서로 변환
char_input_ids = torch.tensor([char_input_ids])  # 2차원 텐서로 변환
attention_mask = torch.ones_like(char_input_ids)  # 모든 토큰에 대해 마스크 생성

# 저장된 모델 로드
model = AutoModelForTokenClassification.from_pretrained('./MODEL/koelectra_v3_ner_model2')
model.eval()

# 추론
with torch.no_grad():
    outputs = model(input_ids=char_input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # 가장 높은 확률을 가진 클래스 선택

In [ ]:
# 토큰과 예측된 레이블 연결
# tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
predicted_labels = predictions[0].cpu().numpy()

# 결과 출력
for token, label in zip(text_token, predicted_labels):
    print(f"Token: {token} Label: {label}")

Token: 치 Label: 0
Token: 밥 Label: 1
Token: 의 Label: 2
Token: 달 Label: 2
Token: 인 Label: 2
Token: 에 Label: 0
Token: 서 Label: 0
Token: 불 Label: 3
Token: 향 Label: 4
Token: 바 Label: 4
Token: 베 Label: 4
Token: 큐 Label: 4
Token: 치 Label: 4
Token: 밥 Label: 4
Token: 도 Label: 4
Token: 시 Label: 4
Token: 락 Label: 4
Token: 3 Label: 5
Token: 개 Label: 6
Token: , Label: 0
Token: 담 Label: 3
Token: 백 Label: 4
Token: 한 Label: 5
Token: 숯 Label: 6
Token: 불 Label: 4
Token: 간 Label: 4
Token: 장 Label: 4
Token: 치 Label: 4
Token: 밥 Label: 4
Token: 2 Label: 5
Token: 개 Label: 6
Token: 주 Label: 0
Token: 문 Label: 0
Token: 할 Label: 0
Token: 게 Label: 0
